Goal is to translate data from Pixelav (which comes in the form of 8x32 array) into something the board can understand (string of bits 00, 01, 10, 11 that corresponds to different charge thresholds). Then want to map it into array1.
\
\
**questions:**
- what are the different arrays 0,1,2 / what are we planning on filling them with 
- how do you take a time stamp? i don't actually know what pixelav is putting out so i don't know where that comes from

pixelav: 0:3 y, 0:15 x (8x32 --> 256 pixels). then each pixel goes to 2 bits which classify its charge. 

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# pretend pixelav output, based on figure 5 from the testing document
# this will be replaced by an actual pixelav file, but currently there is a size mismatch (we want 8x32 for the chip and the pixelav .out files give
# 13x21)... this is the next thing to work out. 
empty_array = np.zeros((4, 16))
pixelav = np.zeros((8, 32))
count = np.arange(0,4*16,1)
pixels = []

for pixel in range(len(count)):
    start = 0 + (pixel*4)
    mini_array = np.array([[start+2, start], [start+3, start+1]])
    pixels.append(mini_array)
mini_array_idx = 0
for row in range(4):
    for col in range(16):
        row_start = np.abs(6-(4 - 1 - row)*2)
        col_start = (16 - 1 - col)*2
        pixelav[row_start:row_start+2, col_start:col_start+2] = pixels[mini_array_idx]
        mini_array_idx += 1

test_vals = np.random.uniform(low = 400, high = 14000, size=(8,32))
low = 400
thresh1 = 1600
thresh2 = 2400


In [4]:
pixel_data = {}
for row in reversed(range(8)):
    for column in range(32):
        charge = test_vals[row, column]
        pixel = int(pixelav[row, column])
        if charge <= low: 
            label = "00"
        elif low < charge < thresh1:
            label = "01"
        elif thresh1 < charge < thresh2:
            label = "10"
        else:
            label = "11"
        pixel_data[pixel] = {
            'charge': charge,
            'label': label,
            'row': row,
            'column': column
        }

big_row_start = 0
bit_string_list = []

while big_row_start < 8:
    for column in reversed(range(32)):
        for row in range(2):
            overall_row = row + big_row_start
            pixel = pixelav[overall_row, column]
            bit_string_list.append(pixel_data[pixel]['label'])
    big_row_start +=2
bit_string = ''.join(reversed(bit_string_list))

In [5]:
ar1row = 256
ar1col = 16
array1 = np.zeros((ar1row,ar1col))
len_shiftreg = (67+1)*16+4
len_bits = len(bit_string)
len_dnn = (255-100)*16+4*3

# fake numbers for shift register and dnn parts of the array
shiftreg = [0]*len_shiftreg
dnn = [0]*len_dnn

array1_flat = array1.flatten()

i = 0
for row in range(ar1row):
    for col in range(ar1col):
        if i < len_shiftreg:
            array1[row,col] = shiftreg[i]
        elif len_shiftreg <= i < len_shiftreg + len_bits:
            array1[row, col] = bit_string[i - len_shiftreg]
        elif i >= len_shiftreg+len_bits:
            array1[row, col] = dnn[i - (len_shiftreg+len_bits)]
        i+=1

In [9]:
print(np.shape(array1))

(256, 16)
